<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Input, \
    Activation, Conv2D, MaxPooling2D, Conv2DTranspose, Add, concatenate, Concatenate, UpSampling2D

# def bn_Conv2d(x, filters=16, kernel_size=(3, 3), padding='same', strides=(1, 1), dilation_rate=(2, 2), name='Conv'):
#     y = Conv2D(filters=filters, kernel_size=kernel_size, kernel_initializer='he_normal', padding=padding,
#                strides=strides, dilation_rate=dilation_rate, name=name)(x)
#     y = BatchNormalization(name='BN_' + name)(y)
#     y = Activation('relu', name='AC_' + name)(y)
#     return y
def bn_Conv2DTranspose(x, filters=16, kernel_size=(3, 3), strides=(2, 2), padding='same', dilation_rate=(1, 1),
                       name='transpose', activation='relu'):
    up1 = Conv2DTranspose(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                          dilation_rate=dilation_rate, name=name)(x)
    up1 = BatchNormalization()(up1)
    up1 = Activation(activation)(up1)
    return up1

kernel_regularizer= tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)

def bn_Conv2d(x, filters=16, kernel_size=(3, 3), padding='same', strides=(1, 1),
              dilation_rate=(1, 1), name='Conv', activation='relu'):
    y = Conv2D(filters=filters, kernel_size=kernel_size,
               kernel_initializer='he_normal',
               bias_initializer='he_normal',
               #kernel_regularizer=kernel_regularizer,
               #activity_regularizer=tf.keras.regularizers.L2(1e-5),
               padding=padding,
               use_bias=False,
               strides=strides, dilation_rate=dilation_rate,)(x)

    y = BatchNormalization()(y)
    y = Activation(activation)(y)
    return y

def ATC_UNET(input_shape):
    inputs = Input(shape=input_shape)
    input_block= inputs
    p=[]
    for i in range(6):
        en = bn_Conv2d(x=input_block, filters=2**i,
                       kernel_size=(3,3), name='Enc_{0}_1'.format(i+1), dilation_rate=(1,1))
        en = bn_Conv2d(x=en, filters=2 ** (i+1),
                       kernel_size=(3,3), name='Enc_{0}_2'.format(i + 1), dilation_rate=(2,2))
        en = bn_Conv2d(x=en, filters=2 ** (i+2),
                       kernel_size=(3,3), name='Enc_{0}_3'.format(i + 1), dilation_rate=(3,3))
        p.append(MaxPooling2D(pool_size=(2,2))(en))
        input_block=p[i]
    trans = bn_Conv2d(x=en, filters=128,
                      kernel_size=(3,3), strides=(2,2),
                      dilation_rate=(1,1), name='transfer')
    de = trans
    for i in range(5):
        de= bn_Conv2DTranspose(x=de, filters=2**(6-i), kernel_size=(3,3), name='Dec_{0}'.format(i+1))
        de= Add()([de, p[4-i]])
    outputs = bn_Conv2DTranspose(x=de, filters=1, kernel_size=(3,3), name='Dec_6', activation='sigmoid')
    model = Model(inputs=[inputs], outputs=[outputs], name='ATC_UNET')
    return model

def New_ATC_UNET(input_shape):
    inputs = Input(shape=input_shape)
    input_block= inputs
    p=[]
    for i in range(6):
        en1 = bn_Conv2d(x=input_block, filters=2**i,
                        kernel_size=(3,3), name='Enc_{0}_1'.format(i+1), dilation_rate=(1,1))
        en2 = bn_Conv2d(x=input_block, filters=2 ** (i+1),
                        kernel_size=(3,3), name='Enc_{0}_2'.format(i + 1), dilation_rate=(2,2))
        en3 = bn_Conv2d(x=input_block, filters=2 ** (i+2),
                        kernel_size=(3,3), name='Enc_{0}_3'.format(i + 1), dilation_rate=(3,3))
        en = concatenate(inputs=[en1, en2, en3],axis=-1)
        en= bn_Conv2d(x=en, filters=2 ** (i+2),
                      kernel_size=(1,1), dilation_rate=(1,1), name='Enc_add_{0}'.format(i+1))
        p.append(MaxPooling2D(pool_size=(2,2))(en))

        input_block=p[i]
    trans = bn_Conv2d(x=en, filters=128, kernel_size=(3,3), strides=(2,2), dilation_rate=(1,1), name='transfer')
    de = trans
    for i in range(5):
        de= bn_Conv2DTranspose(x=de, filters=2**(6-i), kernel_size=(3,3), name='Dec_{0}'.format(i+1))
        de= Add()([de, p[4-i]])
    outputs = bn_Conv2DTranspose(x=de, filters=1, kernel_size=(3,3), name='Dec_6', activation='sigmoid')
    model = Model(inputs=[inputs], outputs=[outputs], name='ATC_UNET')
    return model

def AT_Unet_2(input_shape):
    inputs = Input(shape=input_shape, name='Input_layer')
    input_block = inputs
    n=5
    filters = [2, 4, 8, 16, 32, 128]
    p=[]
    for i in range(n):
        en = bn_Conv2d(x=input_block, filters=filters[i],
                       kernel_size=(3,3), name='Enc_{0}_1'.format(i+1))
        en= bn_Conv2d(x = en, filters=filters[i], kernel_size=(3,3),
                      dilation_rate=(2,2),
                      name='Enc_{0}_2'.format(i+1))
        en = bn_Conv2d(x=en, filters=filters[i], kernel_size=(3, 3),
                       dilation_rate=(3,3),
                       name='Enc_{0}_3'.format(i + 1))
        p.append(MaxPooling2D(pool_size=(2,2))(en))
        input_block = p[i]

    trans= bn_Conv2d(x=en, filters=100, kernel_size=(3,3), strides=(2,2),
                     dilation_rate=(1,1), name='Transfer')
    de=trans

    for i in range(n-1,0,-1):
        de= bn_Conv2DTranspose(x=de, filters=filters[i], kernel_size=(3,3), name='Dec_{0}'.format(i+1))
        de = concatenate(axis=-1, inputs=[de, p[i-1]])

    outputs = bn_Conv2DTranspose(x=de, filters=1, kernel_size=(3,3), name='Dec_last', activation='sigmoid')
    model= Model(inputs=[inputs], outputs=[outputs], name='AT_UNET')

    return model

def inception_block(x, filters):
    # 5x5 brand
    brand5x5 = bn_Conv2d(x,kernel_size=(1,1), filters=1, activation='relu')
    brand5x5 = bn_Conv2d(brand5x5, kernel_size=(5,5), filters=filters, strides=(2,2), activation='relu')
    #  3x3 brand
    brand3x3 = bn_Conv2d(x,kernel_size=(1,1), filters=1, activation='relu')
    brand3x3 =  bn_Conv2d(brand3x3, kernel_size=(3,3), filters=filters, strides=(2,2), activation='relu')
    # Pooling brand
    brandpool = MaxPooling2D(pool_size=(2,2)) (x)
    brandpool = bn_Conv2d(brandpool, kernel_size=(1, 1), filters=1, activation='relu')
    # 1x1 brand
    brand1x1 = bn_Conv2d(x,kernel_size=(1,1), filters=filters, strides=(2,2), activation='relu')
    output = Add()([brand5x5, brand3x3, brandpool, brand1x1])
    #output = Concatenate(axis=-1)([brand5x5, brand3x3, brandpool, brand1x1])
    return (output, brand1x1)


def new_inception_block(x, filters):
    # 5x5 brand
    brand5x5 = bn_Conv2d(x,kernel_size=(1,1), filters=1, activation='relu')
    brand5x5 = bn_Conv2d(brand5x5, kernel_size=(3,3), filters=filters, dilation_rate=(2,2),
                         strides=(1,1), activation='relu')
    brand5x5 = MaxPooling2D(pool_size=(2,2))(brand5x5)

    #  3x3 brand
    brand3x3 = bn_Conv2d(x,kernel_size=(1,1), filters=1, activation='relu')
    brand3x3 =  bn_Conv2d(brand3x3, kernel_size=(3,3), filters=filters, dilation_rate=(1,1),
                          strides=(1,1), activation='relu')
    brand3x3 = MaxPooling2D(pool_size=(2, 2))(brand3x3)

    # Pooling brand
    brandpool = bn_Conv2d(x, kernel_size=(1, 1), filters=1, activation='relu')
    brandpool = MaxPooling2D(pool_size=(2,2)) (brandpool)

    # 1x1 brand
    brand1x1 = bn_Conv2d(x,kernel_size=(1,1), filters=filters, strides=(2,2), activation='relu')
    #output = Concatenate(axis=-1)([brand5x5, brand3x3])
    output = Add()([brand5x5, brand3x3])
    return (output, output)

def AT_Unet_Inception(input_shape):
    # 1st layer, don't change size

    input = Input(shape=input_shape, name='Input_layer')
    coder=input
    # coder =  bn_Conv2d(x=coder, filters=1, kernel_size=(3,3),
    #                    padding='same', strides=(1,1),
    #                    dilation_rate=(1,1), activation='relu'
    #                    )
    n = 6

    filters = [5, 5, 5, 5, 5, 5]
    pass_f = filters[n-1]

    p=[]
   # coder = bn_Conv2d(x=coder, filters=1, kernel_size=(3, 3), padding='same')

    for i in range(n):
        coder, link = new_inception_block(coder, filters=filters[i])
        p.append(link)

    decoder = bn_Conv2d(x=coder, kernel_size=(3, 3), padding='same', filters=pass_f, activation='relu')
    # define Decoder
    for i in range(n-1, -1, -1):
        decoder = Add()([decoder, p[i]])
        decoder = bn_Conv2d(x=decoder, filters=filters[i],
                            kernel_size=(3,3), activation='relu', padding='same')
        decoder = UpSampling2D(size=(2, 2))(decoder)

    output = bn_Conv2d(x=decoder, filters=1, kernel_size=(3,3), activation='sigmoid', padding='same')
    model = Model(input, output, name='output_generator')
    model.summary()
    return model


